# NLP. Определение оценки отзыва по содержанию.

Нам предоставленны данные, содержащие отзывы и оценки разных банков. Наша задача стоит в том, чтобы модель обучилась предсказывать оценку, от 1 до 5, по содержанию отзыва.

## Обзор данных.

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('C:/Users/vseli/Downloads/hse-nlp-bootcamp/train_ml.csv')

In [3]:
data.head()

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,13.12.2016 1:05
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,28.06.2019 13:54
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,15.07.2020 14:54
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,04.08.2020 6:38


In [4]:
data.isna().sum()

bank          0
feeds         0
grades    23524
date          0
dtype: int64

In [5]:
data = data.dropna()
data.isna().sum()

bank      0
feeds     0
grades    0
date      0
dtype: int64

In [6]:
data['feeds'] = data['feeds'].str.lower()

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51476 entries, 0 to 74999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   bank    51476 non-null  object 
 1   feeds   51476 non-null  object 
 2   grades  51476 non-null  float64
 3   date    51476 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


## Подготовка данных для обучения модели.

In [8]:
def clear_text(text):
    clear_text = re.sub(r'[^а-яА-ЯёЁ ]', ' ', text)
    clear_text = clear_text.split()
    clear_text = " ".join(clear_text)
    return clear_text

In [9]:
data['feeds'] = data['feeds'].apply(str)
data['feeds'] = data.feeds.apply(clear_text)

In [10]:
features_train = data['feeds']

In [11]:
features_train

0        много лет являюсь клиентом этого банка но посл...
1        г ростов на дону ул ленина были г с часов до в...
4        имею потребительский кредит взятый в связь бан...
7        хочу выразить глубокую благодарность всем сотр...
8        марта г в при проходе в метро курская по социа...
                               ...                        
74995    три года длится эпопея с закрытием кредитной к...
74996    являлся клиентом бинбанка пока после слияния н...
74997    мая г мною вносились деньги на банковскую карт...
74998    здравствуйте сегодня захожу в мобильный банк а...
74999    уважаемое руководство банка если все услуги ва...
Name: feeds, Length: 51476, dtype: object

In [12]:
target_train = data['grades']

In [13]:
features_train,features_valid,target_train,target_valid = train_test_split(features_train,target_train,test_size=0.2)

In [14]:
vec = TfidfVectorizer(ngram_range=(2, 2)) # используем уни-, биграммы
features_train = vec.fit_transform(features_train)
features_valid = vec.transform(features_valid)

In [15]:
features_valid

<10296x2254189 sparse matrix of type '<class 'numpy.float64'>'
	with 1151271 stored elements in Compressed Sparse Row format>

## Обучение моделей.

__Логистическая регрессия__

In [16]:
model = LogisticRegression(random_state=56789)
model.fit(features_train,target_train)

C:\Users\vseli\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=56789)

In [17]:
pred = model.predict(features_valid)
f1_score(target_valid,pred,average='micro')

0.753982128982129

## Предсказания для тестовых данных.

In [18]:
data_test = pd.read_csv('C:/Users/vseli/Downloads/hse-nlp-bootcamp/new_test_ml.csv')
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17220 entries, 0 to 17219
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  17220 non-null  int64 
 1   bank        17220 non-null  object
 2   feeds       17220 non-null  object
 3   date        17220 non-null  object
dtypes: int64(1), object(3)
memory usage: 538.2+ KB


In [19]:
data_test['feeds'] = data_test['feeds'].str.lower()

In [20]:
data_test['feeds'] = data_test['feeds'].apply(str)
data_test['feeds'] = data_test.feeds.apply(clear_text)

In [21]:
feature_test = data_test['feeds']
feature_test = vec.transform(feature_test)
feature_test

<17220x2254189 sparse matrix of type '<class 'numpy.float64'>'
	with 1932620 stored elements in Compressed Sparse Row format>

In [109]:
pred_test = model.predict(feature_test)
pred_test

array([1., 1., 1., ..., 1., 1., 1.])

In [110]:
pred_test = pd.DataFrame(pred_test)
pred_test = pred_test.astype(int)

In [111]:
filename = 'submission.csv'
pred_test.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: submission.csv


In [112]:
subm = pd.read_csv('C:/Users/vseli/Downloads/submission.csv')

In [113]:
subm = subm.rename(columns={"0": "grades"})

In [114]:
subm.index.name = 'inds'

In [115]:
subm

,grades
inds,
0,1
1,1
2,1
3,1
4,1
...,...
17215,1
17216,1
17217,1


In [116]:
filename = 'submission_me.csv'
subm.to_csv(filename,index=True)
print('Saved file: ' + filename)

Saved file: submission_me.csv
